<a href="https://colab.research.google.com/github/HarrisDelis/CompPhysics/blob/master/Quantum_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install libraries
!pip install tensorflow
!pip install pennylane==0.33.1
!pip install PennyLane_Lightning

In [3]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [4]:
n_epochs = 60
n_train = 1000
n_test = 400

Save_Path = "../_static/demostration_assets/quanvolution/" #Data saving folder
Preprocess = True           #If False, skip quantum processing and load data from Save_Path
np.random.seed(0)           #Seed for NumPy random number generator
tf.random.set_seed(0)       #Seed for TensorFlow random number generator

In [6]:
#Download MNIST Dataset
Data = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = Data.load_data()

X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_test = X_test[:n_train]
y_test = y_test[:n_train]

X_train = X_train / 255.0
X_test = X_test / 255.0

#Add extra dimension for convolution channels
X_train = np.array(X_train[..., tf.newaxis], requires_grad=False)
X_test = np.array(X_test[..., tf.newaxis], requires_grad=False)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
dev = qml.device("lightning.qubit", wires = 16)

#Convolution Block (CB)

def conv(params, i, qubits=[0,1,2,3]):

  qml.RY(params[i], wires=qubits[0])
  qml.RY(params[i+1], wires=qubits[1])
  qml.RY(params[i+2], wires=qubits[2])
  qml.RY(params[i+3], wires=qubits[3])
  qml.CNOT(wires=[qubits[3], qubits[0]])
  qml.CNOT(wires=[qubits[2], qubits[3]])
  qml.CNOT(wires=[qubits[1], qubits[2]])
  qml.CNOT(wires=[qubits[0], qubits[1]])
  qml.RY(params[i+4], wires=qubits[0])
  qml.RY(params[i+5], wires=qubits[1])
  qml.RY(params[i+6], wires=qubits[2])
  qml.RY(params[i+7], wires=qubits[3])
  qml.CNOT(wires=[qubits[3], qubits[2]])
  qml.CNOT(wires=[qubits[0], qubits[3]])
  qml.CNOT(wires=[qubits[1], qubits[0]])
  qml.CNOT(wires=[qubits[2], qubits[1]])

In [9]:
# Pooling Block (PB)

def pool(params, i, qubits=[0,1,2,3]):

  qml.RY(params[i], wires=qubits[0])
  qml.RZ(params[i+1], wires=qubits[1])
  qml.RY(params[i+2], wires=qubits[2])
  qml.RZ(params[i+3], wires=qubits[3])
  qml.CNOT(wires=[qubits[3], qubits[2]])
  qml.CNOT(wires=[qubits[2], qubits[1]])
  qml.CNOT(wires=[qubits[1], qubits[0]])
  qml.RZ(params[i+4], wires=qubits[0])
  qml.RY(params[i+5], wires=qubits[1])
  qml.RZ(params[i+6], wires=qubits[2])
  qml.PauliZ(qubits[3])

In [10]:
# Parameterized Quantum Circuit (PQC)

def parameterized_circuit(params):

  conv(params, 0, qubits=[0,1,2,3])
  conv(params, 0, qubits=[4,5,6,7])
  conv(params, 0, qubits=[8,9,10,11])
  conv(params, 0, qubits=[12,13,14,15])
  conv(params, 0, qubits=[2,3,4,5])
  conv(params, 0, qubits=[6,7,8,9])
  conv(params, 0, qubits=[10,11,12,13])
  conv(params, 0, qubits=[4,5,6,7])
  conv(params, 0, qubits=[8,9,10,11])

  pool(params, 8, qubits=[0,1,2,3])
  pool(params, 8, qubits=[4,5,6,7])
  pool(params, 8, qubits=[8,9,10,11])
  pool(params, 8, qubits=[12,13,14,15])

  conv(params, 15, qubits=[0,1,2,4])
  conv(params, 15, qubits=[5,6,8,9])
  conv(params, 15, qubits=[10,12,13,14])
  conv(params, 15, qubits=[2,4,5,6])
  conv(params, 15, qubits=[8,9,10,12])

  pool(params, 23, qubits=[0,1,2,4])
  pool(params, 23, qubits=[5,6,8,9])
  pool(params, 23, qubits=[10,12,13,14])

  conv(params, 30, qubits=[1,2,5,6])
  conv(params, 30, qubits=[8,10,12,13])

  pool(params, 38, qubits=[1,2,5,6])
  pool(params, 38, qubits=[8,10,12,13])

In [13]:
# Initialize parameters (0,1)
num_params = 45
params = np.random.rand(num_params)

#Angle Embedding and PQC

@qml.qnode(dev)
def q(input):
  for i in range(16):
    qml.RY(input[i], wires=i)
  parameterized_circuit(params)

  return [qml.expval(qml.PauliZ(j)) for j in [0,1,2,5,8,10,12]]

In [14]:
n = np.random.rand(16)
print(n)

print(params)

k = q(n)
print(k)

f = qml.draw(q)(n)
print(f)

[0.59204193 0.57225191 0.22308163 0.95274901 0.44712538 0.84640867
 0.69947928 0.29743695 0.81379782 0.39650574 0.8811032  0.58127287
 0.88173536 0.69253159 0.72525428 0.50132438]
[0.11037514 0.65632959 0.13818295 0.19658236 0.36872517 0.82099323
 0.09710128 0.83794491 0.09609841 0.97645947 0.4686512  0.97676109
 0.60484552 0.73926358 0.03918779 0.28280696 0.12019656 0.2961402
 0.11872772 0.31798318 0.41426299 0.0641475  0.69247212 0.56660145
 0.26538949 0.52324805 0.09394051 0.5759465  0.9292962  0.31856895
 0.66741038 0.13179786 0.7163272  0.28940609 0.18319136 0.58651293
 0.02010755 0.82894003 0.00469548 0.67781654 0.27000797 0.73519402
 0.96218855 0.24875314 0.57615733]
[array(0.01060201), array(0.01004935), array(-0.00957012), array(-0.00063868), array(0.02439781), array(-0.00175501), array(0.00971657)]
 0: ──RY(0.59)──RY(0.11)─╭X──────────────╭●─────────RY(0.37)─╭●─╭X─────────RY(0.10)────────────────
 1: ──RY(0.57)──RY(0.66)─│─────╭●────────╰X─────────RY(0.82)─│──╰●────────╭X────

In [16]:
def quanv(image):

  out = np.zeros((7, 7, 7))


  for j in range(0, 28, 4):
    for k in range(0, 28, 4):

      q_results = q(
          [
              image[j, k ,0],
              image[j, k+1, 0],
              image[j, k+2, 0],
              image[j, k+3, 0],
              image[j+1, k, 0],
              image[j+1, k+1, 0],
              image[j+1, k+2, 0],
              image[j+1, k+3, 0],
              image[j+2, k, 0],
              image[j+2, k+1, 0],
              image[j+2, k+2, 0],
              image[j+2, k+3, 0],
              image[j+3, k, 0],
              image[j+3, k+1, 0],
              image[j+3, k+2, 0],
              image[j+3, k+3, 0]
          ]
      )

      for c in range(7):
        out[j // 4, k//4, c] = q_results[c]
    return out

In [ ]:
if Preprocess == True:
  q_X_train = []
  print("Quantum pre-processing of train images:")
  for idx, img in enumerate(X_train):
    if(idx % 10 == 0):
      print(idx)
    q_X_train.append(quanv(img))
  q_X_train = np.asarray(q_X_train)

  q_X_test = []
  print("\nQuantum pre-processing of test images:")
  for idx, img in enumerate(X_test):
    if (idx % 10 == 0):
      print(idx)
    q_X_test.append(quanv(img))
  q_X_test = np.asarray(q_X_test)

  np.save(Save_Path + "q_X_train_4x4.npy", q_X_train)
  np.save(Save_Path + "q_X_test_4x4.npy", q_X_test)

q_X_train = np.load(Save_Path + "q_X_train_4x4.npy")
q_X_test = np.load(Save_Path + "q_X_test_4x4.npy")


In [ ]:
n_samples = 4
n_channels = 7
fig, axes = plt.subplots(1 + n_channels, n_samples, figsize=(10,10))
for k in range(n_samples):
  axes[0, 0].set_ylabel("Input")
  if k != 0:
    axes[0,k].yaxis.set_visible(False)
  axes[0,k].imshow(X_train[k, :, :, 0], cmap="gray")

  #Plot all output channels
  for c in range(n_channels):
    axes[c+1, 0].set_ylabel("Output [ch. {}]".format(c))
    if k != 0:
      axes[c, k].yaxis.set_visible(False)
    axes[c+1, k].imshow(q_X_train[k, :, : c], cmap="gray")

plt.tight_layout()
plt.show()

In [19]:
def MyModel():
  """Initializes and returns a custom Keras model which is ready to be trained"""
  model = keras.models.Sequential([
      keras.layers.Flatten(),
      keras.layers.Dense(10, activation="softmax")
  ])

  model.compile(
      optimizer="adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"]
  )
  return model

In [ ]:
q_model = MyModel()

q_history = q_model.fit(
    q_X_train,
    y_train,
    validation_data=(q_X_test, y_test),
    batch_size=1,
    epochs=n_epochs,
    verbose=2
)

In [ ]:
c_model = MyModel()

c_history = c_model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    batch_size=4,
    epochs=n_epochs,
    verbose=2
)

In [ ]:
plt.style.use("seaborn")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6,9))

ax1.plot(q_history.history["val_accuracy"], "-ob", label="With quantum layer")
ax1.plot(c_history.history["val_accuracy"], "-ob", label="Without quantum layer")
ax1.set_ylabel("Accuracy")
ax1.set_ylim([0,1])
ax1.set_xlabel("Epoch")
ax1.legend()

ax2.plot(q_history.history["val_loss"], "-ob", label="With quantum layer")
ax2.plot(c_history.history["val_loss"], "-ob", label="Without quantum layer")
ax2.set_ylabel("Loss")
ax2.set_ylim(top=2.5)
ax2.set_xlabel("Epoch")
ax2.legend()
plt.tight_layout()
plt.show()